In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
lmap = lambda funcion, iterable: list(map(funcion, iterable))
lfilter = lambda funcion, iterable: list(filter(funcion, iterable))

In [3]:
dir_dict = dict()

dir_dict["data"] = "../data/"
dir_dict["raw"] = os.path.join(dir_dict["data"], "raw")
dir_dict["mid"] = os.path.join(dir_dict["data"], "mid")
dir_dict["raw_csv"] = os.path.join(dir_dict["raw"], "csv")
dir_dict["raw_json"] = os.path.join(dir_dict["raw"], "json")
dir_dict["html"] = os.path.join(dir_dict["raw"], "html")
dir_dict["completed_html"] = os.path.join(dir_dict["html"], "completed")
dir_dict["upcoming_html"] = os.path.join(dir_dict["html"], "upcoming")
dir_dict["fighterlist_html"] = os.path.join(dir_dict["html"], "fighterlist")
dir_dict["fighters_html"] = os.path.join(dir_dict["html"], "fighters")
dir_dict["completed_eventlist_html"] = os.path.join(dir_dict["completed_html"], "eventlist")
dir_dict["completed_events_html"] = os.path.join(dir_dict["completed_html"], "events")
dir_dict["completed_fights_html"] = os.path.join(dir_dict["completed_html"], "fights")
dir_dict["upcoming_eventlist_html"] = os.path.join(dir_dict["upcoming_html"], "eventlist")
dir_dict["upcoming_events_html"] = os.path.join(dir_dict["upcoming_html"], "events")
dir_dict["upcoming_fights_html"] = os.path.join(dir_dict["upcoming_html"], "fights")

In [4]:
for folderpath in dir_dict.values():
    os.makedirs(folderpath, exist_ok=True)

In [5]:
filepath = os.path.join(dir_dict["raw_json"], "completed_fights_data.json")
with open(filepath, "r") as f:
    completed_fights_lod = json.load(f)

In [6]:
len(completed_fights_lod)

6875

In [7]:
def process_fight_dict(fight_dict):

    def merge_additional_data(data_header):
        additional_dict = fight_dict.pop(data_header)
        fighter_names = additional_dict["Fighter"]
        round_order = lmap(lambda x: x.replace(" ", ""), additional_dict["Round"])
        if (fighter_names[0] == fight_dict["Fighter1 Name"]) and (fighter_names[1] == fight_dict["Fighter2 Name"]):
            fighter_order = ["Fighter1", "Fighter2"]
        elif (fighter_names[0] == fight_dict["Fighter2 Name"]) and (fighter_names[1] == fight_dict["Fighter1 Name"]):
            fighter_order = ["Fighter1", "Fighter2"]
        else:
            raise Exception(f"Fighter names not congurent in {data_header} table")

        for key, lov in list(additional_dict.items())[1:-1]:
            for i, v in enumerate(lov):
                var = f"{key}_SS" if data_header == "Significant Strikes" else key
                fight_dict[f"{var}_{round_order[i]}_{fighter_order[(i % 2)]}"] = v

    fight_dict_keys = fight_dict.keys()
    additional_data_headers = ["Totals", "Significant Strikes"]
    for data_header in additional_data_headers:
        if data_header in fight_dict_keys:
            merge_additional_data(data_header)

    return fight_dict

In [8]:
res_fight_dict_list = lmap(process_fight_dict, completed_fights_lod)

In [9]:
len(res_fight_dict_list)

6875

In [10]:
fights_df = pd.DataFrame(res_fight_dict_list)

In [11]:
fights_df.memory_usage(deep=True).sum() / 10**6

77.408445

In [12]:
filepath = os.path.join(dir_dict["raw_csv"], "completed_fights_data.csv")
fights_df.to_csv(filepath, index=False)

In [13]:
fights_df = pd.read_csv(filepath)

/home/alancmathew/miniconda3/envs/ufc-fight-predictor/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (223) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
fights_df["Event ID"] = fights_df["Event Url"].map(lambda s: os.path.split(s)[1])

In [15]:
fights_df["Fighter1 ID"] = fights_df["Fighter1 Url"].map(lambda s: os.path.split(s)[1])
fights_df["Fighter2 ID"] = fights_df["Fighter2 Url"].map(lambda s: os.path.split(s)[1])

In [16]:
fights_df = fights_df.drop(columns=["Event Url", "Fighter1 Url", "Fighter2 Url"])

In [17]:
fights_df.head()

,Event Name,Fight ID,Fighter1 Status,Fighter1 Name,Fighter2 Status,Fighter2 Name,Bout,Method,Round,Time,...,Ground_SS_Round4_Fighter2,Ground_SS_Round5_Fighter1,Ground_SS_Round5_Fighter2,Details: Point Deducted,Details: Two Points Deducted,Details: Technical Decision,Details: Points Deducted,Event ID,Fighter1 ID,Fighter2 ID
0,UFC Fight Night: Covington vs. Woodley,859ca56770ac7672,W,Mackenzie Dern,L,Randa Markos,Women's Strawweight Bout,Submission,1,3:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,831b937811804dad,7447e9f28508106a,4a57ebb14315b251
1,UFC Fight Night: Barboza vs. Lee,6d8ce1abde550bc6,W,Frankie Edgar,L,Cub Swanson,Featherweight Bout,Decision - Unanimous,3,5:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ad99fa5325519169,f2688492b9a525a3,d247691a6c0e9034
2,UFC on FX: Sotiropoulos vs Pearson,d4c85298623130de,W,Mike Pierce,L,Seth Baczynski,Welterweight Bout,Decision - Unanimous,3,5:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e8c170a64dc920ac,236a37d96d476164,fd5b6598a3b70c0a
3,UFC Fight Night: Cejudo vs. Dillashaw,0261adcfc6e49150,W,Gregor Gillespie,L,Yancy Medeiros,Lightweight Bout,KO/TKO,2,4:59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d4da8995fc91e7ef,84ff027394f7e470,813550bc53b15fb0
4,UFC Fight Night: Barboza vs. Chikadze,089a17e450cc98aa,L,Kevin Lee,W,Daniel Rodriguez,Welterweight Bout,Decision - Unanimous,3,5:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0db9d2486d564a3c,ee9ebceabfd16fa7,8a1f3b5c526cd6e6


### Join Weight Class data

In [18]:
filepath = os.path.join(dir_dict["raw_csv"], "completed_fight_urls.csv")
fights_df2 = pd.read_csv(filepath)

In [19]:
fights_df2["Fight ID"] = fights_df2["Fight Url"].map(lambda s: os.path.split(s)[1])
fights_df2 = fights_df2.drop(columns=["Fight Url"])

In [20]:
fights_df2.head()

,Weight Class,Fight ID
0,Lightweight,81f287d035190506
1,Heavyweight,1f0254eeb2025e4f
2,Lightweight,4942323a5d5a6594
3,Women's Strawweight,95ead23dc0c73285
4,Featherweight,164f56eb25d6a9b4


In [21]:
fights_df = fights_df.merge(fights_df2, on="Fight ID", how="left")

In [22]:
fights_df["Weight Class"].isnull().sum()

0

### Join Events data

In [23]:
filepath = os.path.join(dir_dict["raw_csv"], "completed_events.csv")
events_df = pd.read_csv(filepath)

In [24]:
events_df.head()

,name,date,location,url
0,UFC Fight Night: Cannonier vs. Strickland,"December 17, 2022","Las Vegas, Nevada, USA",http://ufcstats.com/event-details/56ec58954158...
1,UFC 282: Blachowicz vs. Ankalaev,"December 10, 2022","Las Vegas, Nevada, USA",http://ufcstats.com/event-details/f65a0eb902f9...
2,UFC Fight Night: Thompson vs. Holland,"December 03, 2022","Orlando, Florida, USA",http://ufcstats.com/event-details/b23388ff8ac6...
3,UFC Fight Night: Nzechukwu vs. Cutelaba,"November 19, 2022","Las Vegas, Nevada, USA",http://ufcstats.com/event-details/012fc7cd0779...
4,UFC 281: Adesanya vs. Pereira,"November 12, 2022","New York City, New York, USA",http://ufcstats.com/event-details/b3b6e80b7d5f...


In [25]:
events_df["Event ID"] = events_df["url"].map(lambda s: os.path.split(s)[1])
events_df = events_df.drop(columns=["name", "url"])

In [26]:
events_df.head()

,date,location,Event ID
0,"December 17, 2022","Las Vegas, Nevada, USA",56ec58954158966a
1,"December 10, 2022","Las Vegas, Nevada, USA",f65a0eb902f9476b
2,"December 03, 2022","Orlando, Florida, USA",b23388ff8ac6637b
3,"November 19, 2022","Las Vegas, Nevada, USA",012fc7cd0779c09a
4,"November 12, 2022","New York City, New York, USA",b3b6e80b7d5f8f0d


In [27]:
fights_df = fights_df.merge(events_df, on="Event ID", how="left")

In [28]:
fights_df.iloc[:,-2:].isnull().sum()

date        0
location    0
dtype: int64

In [29]:
fights_df.head()

,Event Name,Fight ID,Fighter1 Status,Fighter1 Name,Fighter2 Status,Fighter2 Name,Bout,Method,Round,Time,...,Details: Point Deducted,Details: Two Points Deducted,Details: Technical Decision,Details: Points Deducted,Event ID,Fighter1 ID,Fighter2 ID,Weight Class,date,location
0,UFC Fight Night: Covington vs. Woodley,859ca56770ac7672,W,Mackenzie Dern,L,Randa Markos,Women's Strawweight Bout,Submission,1,3:44,...,NaN,NaN,NaN,NaN,831b937811804dad,7447e9f28508106a,4a57ebb14315b251,Women's Strawweight,"September 19, 2020","Las Vegas, Nevada, USA"
1,UFC Fight Night: Barboza vs. Lee,6d8ce1abde550bc6,W,Frankie Edgar,L,Cub Swanson,Featherweight Bout,Decision - Unanimous,3,5:00,...,NaN,NaN,NaN,NaN,ad99fa5325519169,f2688492b9a525a3,d247691a6c0e9034,Featherweight,"April 21, 2018","Atlantic City, New Jersey, USA"
2,UFC on FX: Sotiropoulos vs Pearson,d4c85298623130de,W,Mike Pierce,L,Seth Baczynski,Welterweight Bout,Decision - Unanimous,3,5:00,...,NaN,NaN,NaN,NaN,e8c170a64dc920ac,236a37d96d476164,fd5b6598a3b70c0a,Welterweight,"December 14, 2012","Gold Coast, Queensland, Australia"
3,UFC Fight Night: Cejudo vs. Dillashaw,0261adcfc6e49150,W,Gregor Gillespie,L,Yancy Medeiros,Lightweight Bout,KO/TKO,2,4:59,...,NaN,NaN,NaN,NaN,d4da8995fc91e7ef,84ff027394f7e470,813550bc53b15fb0,Lightweight,"January 19, 2019","Brooklyn, New York, USA"
4,UFC Fight Night: Barboza vs. Chikadze,089a17e450cc98aa,L,Kevin Lee,W,Daniel Rodriguez,Welterweight Bout,Decision - Unanimous,3,5:00,...,NaN,NaN,NaN,NaN,0db9d2486d564a3c,ee9ebceabfd16fa7,8a1f3b5c526cd6e6,Welterweight,"August 28, 2021","Las Vegas, Nevada, USA"


### Join Fighters data

In [42]:
filepath = os.path.join(dir_dict["raw_csv"], "fighters_data.csv")
fighters1_df = pd.read_csv(filepath)
fighters1_df.columns = [f"Fighter1 {col}" if "Fighter" not in col else col.replace("Fighter", "Fighter1") 
                        for col in fighters1_df.columns]

In [43]:
fighters1_df.head()

,Fighter1 Name,Fighter1 ID,Fighter1 Record,Fighter1 Nickname,Fighter1 Height,Fighter1 Weight,Fighter1 Reach,Fighter1 STANCE,Fighter1 DOB,Fighter1 SLpM,Fighter1 Str. Acc.,Fighter1 SApM,Fighter1 Str. Def,Fighter1 TD Avg.,Fighter1 TD Acc.,Fighter1 TD Def.,Fighter1 Sub. Avg.
0,Tamires Vidal,4f3064faf17aa711,7-1-0,Tractor,"5'6""",135 lbs.,"68""",Orthodox,"Jul 27, 1998",5.48,58%,4.19,7%,0.00,0%,0%,0.0
1,Tatsuya Kawajiri,80d918336163b80c,36-11-2,Crusher,"5'7""",145 lbs.,"69""",Orthodox,"May 08, 1978",2.04,44%,2.01,50%,3.63,48%,63%,1.1
2,Joe Giannetti,946db8a378903cc3,6-1-0,NaN,"6'0""",155 lbs.,"74""",Southpaw,"Jul 05, 1995",1.93,38%,2.80,40%,1.00,16%,0%,0.0
3,Brock Jardine,64959f90b6dd9759,12-6-0,The Machine,"5'11""",170 lbs.,NaN,Orthodox,"Jul 10, 1985",1.06,30%,3.34,56%,0.00,0%,14%,0.8
4,Melissa Martinez,3f4c3bc822bea45d,7-1-0,Super Mely,"5'2""",115 lbs.,"66""",Southpaw,"Aug 18, 1997",3.00,57%,2.53,45%,1.00,50%,0%,0.0


In [44]:
filepath = os.path.join(dir_dict["raw_csv"], "fighters_data.csv")
fighters2_df = pd.read_csv(filepath)
fighters2_df.columns = [f"Fighter2 {col}" if "Fighter" not in col else col.replace("Fighter", "Fighter2") 
                        for col in fighters2_df.columns]

In [45]:
fights_df = fights_df.merge(fighters1_df, on="Fighter1 ID", how="left")
fights_df = fights_df.merge(fighters2_df, on="Fighter2 ID", how="left")

In [47]:
fights_df.head()

,Event Name,Fight ID,Fighter1 Status,Fighter1 Name_x,Fighter2 Status,Fighter2 Name_x,Bout,Method,Round,Time,...,Fighter2 STANCE,Fighter2 DOB,Fighter2 SLpM,Fighter2 Str. Acc.,Fighter2 SApM,Fighter2 Str. Def,Fighter2 TD Avg.,Fighter2 TD Acc.,Fighter2 TD Def.,Fighter2 Sub. Avg.
0,UFC Fight Night: Covington vs. Woodley,859ca56770ac7672,W,Mackenzie Dern,L,Randa Markos,Women's Strawweight Bout,Submission,1,3:44,...,Orthodox,"Aug 10, 1985",2.97,43%,3.10,58%,1.12,28%,52%,0.2
1,UFC Fight Night: Barboza vs. Lee,6d8ce1abde550bc6,W,Frankie Edgar,L,Cub Swanson,Featherweight Bout,Decision - Unanimous,3,5:00,...,Orthodox,"Nov 02, 1983",4.68,50%,3.73,60%,1.09,52%,61%,0.4
2,UFC on FX: Sotiropoulos vs Pearson,d4c85298623130de,W,Mike Pierce,L,Seth Baczynski,Welterweight Bout,Decision - Unanimous,3,5:00,...,Orthodox,"Oct 26, 1981",3.08,31%,3.14,51%,0.89,16%,48%,0.8
3,UFC Fight Night: Cejudo vs. Dillashaw,0261adcfc6e49150,W,Gregor Gillespie,L,Yancy Medeiros,Lightweight Bout,KO/TKO,2,4:59,...,Orthodox,"Sep 07, 1987",4.36,37%,5.67,47%,0.33,50%,73%,0.7
4,UFC Fight Night: Barboza vs. Chikadze,089a17e450cc98aa,L,Kevin Lee,W,Daniel Rodriguez,Welterweight Bout,Decision - Unanimous,3,5:00,...,Southpaw,"Dec 31, 1986",7.42,50%,5.22,56%,0.63,50%,63%,0.1


In [48]:
fights_df.isnull().sum()

Event Name            0
Fight ID              0
Fighter1 Status       0
Fighter1 Name_x       0
Fighter2 Status       0
                     ..
Fighter2 Str. Def     0
Fighter2 TD Avg.      0
Fighter2 TD Acc.      0
Fighter2 TD Def.      0
Fighter2 Sub. Avg.    0
Length: 259, dtype: int64

In [49]:
filepath = os.path.join(dir_dict["mid"], "completed_fights.csv")
fights_df.to_csv(filepath, index=False)